## 2023 0617 mission 30 (T2-7 대체)
### 개인의 신용 관련 정보데이터 : 신용등급(Credit_Score) 를 예측하시오
- 평가기준 macro-f1
- 제출 방식 : test데이터로 예측한 class 1개 컬럼만 csv로 제출 (컬럼명 pred, 파일명 result.csv)

제출 csv 파일형태
```
pred
1
3
2
1
2
1
```

In [1]:
# Data Load
import pandas as pd
import numpy as np
train = pd.read_csv('/kaggle/input/t2-7-dataset/T2-7 데이터/train10000.csv')
test = pd.read_csv('/kaggle/input/t2-7-dataset/T2-7 데이터/test10000.csv')

print(train.shape, test.shape)

(10000, 21) (10000, 20)


## 데이터 유형 및 null 확인


In [2]:
# data info : train, test 모두 결측이 없는 것으로 나옴
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       10000 non-null  float64
 1   Num_of_Delayed_Payment    10000 non-null  float64
 2   Num_Credit_Inquiries      10000 non-null  float64
 3   Credit_Utilization_Ratio  10000 non-null  float64
 4   Credit_History_Age        10000 non-null  float64
 5   Payment_of_Min_Amount     10000 non-null  object 
 6   Amount_invested_monthly   10000 non-null  float64
 7   Monthly_Balance           10000 non-null  float64
 8   Credit_Score              10000 non-null  int64  
 9   Credit_Mix                10000 non-null  object 
 10  Payment_Behaviour         10000 non-null  object 
 11  Age                       10000 non-null  float64
 12  Annual_Income             10000 non-null  float64
 13  Num_Bank_Accounts         10000 non-null  float64
 14  Num_Cre

In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       10000 non-null  float64
 1   Num_of_Delayed_Payment    10000 non-null  float64
 2   Num_Credit_Inquiries      10000 non-null  float64
 3   Credit_Utilization_Ratio  10000 non-null  float64
 4   Credit_History_Age        10000 non-null  float64
 5   Payment_of_Min_Amount     10000 non-null  object 
 6   Amount_invested_monthly   10000 non-null  float64
 7   Monthly_Balance           10000 non-null  float64
 8   Credit_Mix                10000 non-null  object 
 9   Payment_Behaviour         10000 non-null  object 
 10  Age                       10000 non-null  float64
 11  Annual_Income             10000 non-null  float64
 12  Num_Bank_Accounts         10000 non-null  float64
 13  Num_Credit_Card           10000 non-null  float64
 14  Interes

In [4]:
# object 타입은 세가지 : Payment_of_Min_Amount	Credit_Mix	Payment_Behaviour
train.select_dtypes('O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
0,Yes,Bad,High_spent_Medium_value_payments
1,Yes,Bad,High_spent_Small_value_payments
2,Yes,Bad,Low_spent_Large_value_payments
3,NM,Standard,High_spent_Small_value_payments
4,Yes,Bad,High_spent_Medium_value_payments
...,...,...,...
9995,No,Standard,High_spent_Medium_value_payments
9996,Yes,Standard,Low_spent_Small_value_payments
9997,Yes,Standard,High_spent_Medium_value_payments
9998,Yes,Standard,High_spent_Medium_value_payments


In [5]:
# 숫자형 타입: 
train.select_dtypes(exclude = 'O')

# 범주형처럼 보이는 타입: 'Credit_Score'(타겟) 
# >>> 타겟 데이터만 면밀히 확인하고 나머지는 스케일링에 집중하기.

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Amount_invested_monthly,Monthly_Balance,Credit_Score,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
0,56.0,16.0,11.0,35.598217,120.0,229.093478,252.385965,1,15.0,36597.56,8.0,10.0,29.0,5.0,3143.796667,22.49,2963.18,122.900223
1,49.0,23.0,12.0,25.553106,120.0,104.613906,219.105944,1,28.0,32057.30,9.0,8.0,16.0,7.0,2606.441667,1.40,1327.26,164.859426
2,34.0,20.0,6.0,40.039954,174.0,338.626965,251.265589,1,46.0,75868.80,6.0,10.0,32.0,7.0,6074.400000,3.60,1432.71,297.547446
3,21.0,13.0,8.0,25.711678,143.0,116.816864,259.927960,2,46.0,17092.69,7.0,3.0,19.0,7.0,1695.390833,16.40,1417.06,62.794260
4,19.0,13.0,6.0,39.140463,138.0,87.262887,626.212330,1,45.0,81471.96,6.0,6.0,25.0,5.0,6763.330000,27.09,2679.69,202.857783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.0,20.0,1.0,28.311154,358.0,160.363681,317.931124,2,25.0,36661.93,6.0,3.0,10.0,4.0,2915.160833,1.16,314.26,73.221279
9996,32.0,13.0,5.0,39.741948,299.0,278.359264,160.416098,2,48.0,28161.85,8.0,5.0,6.0,4.0,2422.820833,7.72,699.69,93.506721
9997,13.0,9.0,8.0,25.111049,203.0,39.334921,352.515899,2,36.0,39912.86,3.0,4.0,17.0,6.0,3498.071667,15.72,1734.43,197.956347
9998,10.0,13.0,6.0,32.237478,309.0,115.463482,354.829742,2,45.0,26918.38,6.0,5.0,10.0,2.0,2442.198333,5.48,1054.90,43.926610


## 데이터 셋팅

In [6]:
# print(train.head(3))
# print(test.head(3))

# 우리는 크레딧 스코어를 예측해야함
# 
X_train = train.drop(columns = ['Credit_Score'])
y_train = train['Credit_Score']
X_test = test.copy()

print(X_train.shape, y_train.shape, X_test.shape)

(10000, 20) (10000,) (10000, 20)


## EDA

In [7]:
# train 데이터로 타깃과의 상관관계를 본다
round(train.corr()['Credit_Score'], 4)

# Credit_Utilization_Ratio    0.0426
# Total_EMI_per_month        -0.0010
# 외에는 대부분 0.1을 넘는다. 'Total_EMI_per_month'과 'Credit_Utilization_Ratio'은 drop 하기로 한다.

/tmp/ipykernel_20/1070713224.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  round(train.corr()['Credit_Score'], 4)


Delay_from_due_date        -0.4335
Num_of_Delayed_Payment     -0.3746
Num_Credit_Inquiries       -0.4296
Credit_Utilization_Ratio    0.0426
Credit_History_Age          0.3736
Amount_invested_monthly     0.1485
Monthly_Balance             0.1947
Credit_Score                1.0000
Age                         0.1508
Annual_Income               0.2021
Num_Bank_Accounts          -0.3865
Num_Credit_Card            -0.4149
Interest_Rate              -0.4795
Num_of_Loan                -0.3486
Monthly_Inhand_Salary       0.1996
Changed_Credit_Limit       -0.1686
Outstanding_Debt           -0.3843
Total_EMI_per_month        -0.0010
Name: Credit_Score, dtype: float64

## 전처리

In [8]:
# 수치형 확인
X_train.describe(exclude = 'O')

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Amount_invested_monthly,Monthly_Balance,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,21.263100,13.381300,5.79400,32.306216,220.501400,195.497580,400.130732,33.33680,50318.451889,5.382900,5.538000,14.468200,3.525700,4182.906254,10.435633,1428.996096,105.114322
std,14.778976,6.217876,3.86772,5.093599,99.204263,196.150962,211.034108,10.73122,38171.440801,2.566271,2.096186,8.687977,2.441585,3175.891720,6.501954,1158.852422,125.502925
min,0.000000,0.000000,0.00000,20.719745,2.000000,0.000000,0.007760,14.00000,7006.035000,0.000000,0.000000,1.000000,0.000000,303.645417,0.500000,0.230000,0.000000
25%,10.000000,9.000000,3.00000,28.075839,144.000000,74.056997,268.741936,24.00000,19259.307500,4.000000,4.000000,7.000000,2.000000,1626.342083,5.507500,562.340000,28.588769
50%,18.000000,14.000000,6.00000,32.310153,219.000000,130.089440,334.854343,33.00000,36875.140000,5.000000,5.000000,13.000000,3.000000,3073.272917,9.410000,1168.235000,65.971478
75%,28.000000,18.000000,8.00000,36.491502,301.000000,234.960725,470.351899,42.00000,71507.587500,7.000000,7.000000,20.000000,5.000000,5936.870000,14.630000,1960.420000,144.429302
max,67.000000,28.000000,17.00000,47.103409,404.000000,1674.337665,1567.208309,56.00000,179987.280000,10.000000,11.000000,34.000000,9.000000,15167.180000,29.980000,4998.070000,1701.955013


In [9]:
X_train.shape

(10000, 20)

### 1. 스케일링 방법과 컬럼 정하기

In [10]:
# 숫자형만 선택
temp = X_train.describe(exclude = 'O')
temp = temp.loc['max', :]

# (1) 상식적으로 Age, Annual_Income, Monthly_Inhand_Salary의 경우 어느정도 분포가 의미가 있을 것으로 예상
do_stdscale_list = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary']


# 그외의 컬럼은 정규성 검정하여 표준화를 진행할지 정하자
# 모든 컬럼이 정규성을 만족하지 않는 것으로 나왔음 : 그래도 저 셋은 분포로 옮기도록하고 나머지는 ->
# 만개 밖에 되지 않으므로 모두 표준화스케일링 억지로 옮기는 것보다 민맥스나 로버스트 등을 사용하는 것이 낫다고 생각


# from scipy.stats import shapiro as normtest
# all_list = list(temp.index)
# for i in alllist:
#     s, pv = normtest(train[i])
#     print('{} : '.format(i), round(s, 6), '\n > pval : ', round(pv, 6))

In [11]:
# (2) max선택하여 맥스 1천이 넘는 것은 로그스케일링인덱스
# 1000이 넘어서 로그 스케일링이 필요하다고 생각되는 경우

# 정규화 리스트에 포함되지 않은 것들 대상으로
temp2 = temp[temp.index.isin(do_stdscale_list)==False]
do_logscale_list = list(temp2[temp2>1000].index)
# do_logscale_list 
# ['Amount_invested_monthly',
#  'Monthly_Balance',
#  'Outstanding_Debt',
#  'Total_EMI_per_month']

In [12]:
# (3) 나머지 로버스트 스케일링 할 경우
# 정규화temp2 에서 로그스케일링에 포함되지 않은 것들 대상으로
temp3 = temp2[temp2.index.isin(do_logscale_list)==False]
do_rbscale_list = list(temp3[temp3.index.isin(do_stdscale_list)==False].index)
# do_rbscale_list
# ['Delay_from_due_date',
#  'Num_of_Delayed_Payment',
#  'Num_Credit_Inquiries',
#  'Credit_Utilization_Ratio',
#  'Credit_History_Age',
#  'Num_Bank_Accounts',
#  'Num_Credit_Card',
#  'Interest_Rate',
#  'Num_of_Loan',
#  'Changed_Credit_Limit']

In [13]:
# 리스트 개수가 오브젝트를 제외한 컬럼개수들과 같은지 확인 : 17로 동일
print(X_train.select_dtypes(exclude = 'O').shape[1])
print(len(do_stdscale_list)+ len(do_logscale_list) + len(do_rbscale_list))

17
17


In [14]:
do_logscale_list

['Amount_invested_monthly',
 'Monthly_Balance',
 'Outstanding_Debt',
 'Total_EMI_per_month']

In [15]:
# 스케일링
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [16]:
# 표준화
std_scaler = StandardScaler()
# train
X_train[do_stdscale_list] = std_scaler.fit_transform(X_train[do_stdscale_list])
# test
X_test[do_stdscale_list] = std_scaler.transform(X_test[do_stdscale_list])

In [17]:
# 로그 스케일링

# 하기전 확인
# print(X_train.loc[:, do_logscale_list].head(5))
# print(np.log1p(X_train.loc[:, do_logscale_list]).head(5))

# 진행
X_train.loc[:, do_logscale_list] = np.log1p(X_train.loc[:, do_logscale_list])
X_test.loc[:, do_logscale_list] = np.log1p(X_test.loc[:, do_logscale_list])

In [18]:
# 로버스트 스케일링
rb_scaler = RobustScaler()
# train
X_train[do_rbscale_list] = rb_scaler.fit_transform(X_train[do_rbscale_list])
# test
X_test[do_rbscale_list] = rb_scaler.transform(X_test[do_rbscale_list])

In [19]:
# X_train.describe() # 특별히 튀는 값 없이 대부분 10 미만

### 2. 이상치 탐색 - 드랍과 스케일링을 다 하기 때문에 패스

### 3. 상관관계 적은 컬럼 드랍 : 
'Total_EMI_per_month', 'Credit_Utilization_Ratio'

In [20]:
# drop 처리 준비
df = pd.concat([X_train, X_test], axis = 0)
print(X_train.shape, X_test.shape, df.shape)

(10000, 20) (10000, 20) (20000, 20)


In [21]:
df = df.drop(columns = ['Total_EMI_per_month', 'Credit_Utilization_Ratio'])

In [22]:
X_train, X_test = df[:train.shape[0]], df[train.shape[0]:]
print(X_train.shape, X_test.shape, df.shape)

(10000, 18) (10000, 18) (20000, 18)


### 4. object type 확인 및 전처리

In [23]:
list_obj = X_train.select_dtypes(include = 'O').columns
for i in list_obj:
    print('# {}'.format(i))
    print(X_train[i].value_counts().sort_index(), '\n')

# Payment_of_Min_Amount
NM     1175
No     3556
Yes    5269
Name: Payment_of_Min_Amount, dtype: int64 

# Credit_Mix
Bad         2389
Good        3020
Standard    4591
Name: Credit_Mix, dtype: int64 

# Payment_Behaviour
High_spent_Large_value_payments     1627
High_spent_Medium_value_payments    2743
High_spent_Small_value_payments      637
Low_spent_Large_value_payments       556
Low_spent_Medium_value_payments     1021
Low_spent_Small_value_payments      3416
Name: Payment_Behaviour, dtype: int64 



In [24]:
X_train.sample(10)

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Mix,Payment_Behaviour,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt
2932,0.222222,-0.666667,0.2,-0.261146,Yes,6.158554,5.317609,Standard,High_spent_Medium_value_payments,0.527758,0.208249,1.000000,0.333333,0.307692,0.333333,0.239534,0.939435,7.548219
1083,-0.944444,-1.111111,-0.2,0.726115,NM,5.366641,5.429587,Good,High_spent_Medium_value_payments,1.180093,-0.874113,-0.333333,0.000000,-0.692308,-0.333333,-0.786613,0.134831,6.319833
155,0.222222,-0.555556,0.0,0.796178,No,5.193285,6.549548,Standard,High_spent_Large_value_payments,-1.056484,1.505055,0.666667,0.000000,-0.461538,0.333333,1.511326,-0.675253,7.034731
4399,0.777778,-0.444444,-0.6,0.528662,Yes,3.662022,5.999019,Standard,High_spent_Medium_value_payments,-0.683722,-0.611535,0.666667,0.666667,-0.461538,0.333333,-0.522043,0.641272,6.056854
8858,-0.888889,0.444444,-1.2,0.375796,Yes,5.957694,5.624471,Standard,Low_spent_Medium_value_payments,1.180093,0.013853,-0.666667,0.333333,0.307692,-0.666667,-0.008388,-0.329953,6.189721
7601,2.277778,0.333333,1.4,-0.133758,Yes,3.339511,5.689595,Bad,Low_spent_Large_value_payments,0.620948,-1.078788,1.333333,0.333333,1.538462,0.666667,-1.065928,-0.943820,7.827022
205,2.055556,-0.111111,0.6,-0.821656,Yes,4.066978,5.806702,Standard,Low_spent_Small_value_payments,0.807330,-0.881317,1.000000,0.666667,0.769231,-0.333333,-0.928602,-0.005481,7.694598
468,-0.777778,-1.555556,-1.2,0.681529,No,5.075302,7.030949,Good,High_spent_Large_value_payments,1.552855,3.231169,-0.333333,-0.333333,-0.846154,0.333333,3.254047,-0.751987,6.982083
5788,-0.111111,-0.111111,0.4,1.050955,Yes,5.331718,5.666288,Standard,Low_spent_Small_value_payments,0.341376,-0.649422,-0.666667,-0.333333,0.461538,-1.000000,-0.644065,0.281721,6.636183
4839,2.333333,0.888889,0.4,-0.114650,NM,5.844795,4.482362,Bad,High_spent_Large_value_payments,0.527758,-0.467564,0.666667,1.666667,0.461538,2.000000,-0.423502,-0.727871,7.339882


In [25]:
# 각각 어떤 레이블로 인코딩 하는게 좋을지 정하여 수동 레이블링이 필요해보인다. 좋고 나쁨의 등급이 있기 때문.

# Payment_of_Min_Amount : 급액 상환 최소 금액이 있는가 없는 가 인 것 같다. 있다면 성실할 가능성이 높다. 
# 그러나 없어도 상환을 한번에 많이 할 수도 있으며, NM의 의미를 모르기 때문에 원핫 인코딩이 필요해보인다.

# Credit_Mix : Bad, Standard, Good 에 따라 -1, 1, 2로 라벨 인코딩 하도록 한다. 
# Standard와 Good은 어느정도의 차이가 있겠지만,신용에서 나쁜 것은 더 큰 차이가 있기 때문.

# Payment_Behaviour : 소비 유형과 납입의 크기에 따라 정해둔 것 같다. 
# 소비와 관련하여 따로 있지는 않지만, 다중공선성을 일으킬 가능성이 있기 때문에,
# 없는 채로 하고, low우선 다음 large 우선으로 라벨인코딩을 진행하여 한번 더 하면 좋을 수 있겠다.

In [26]:
# Payment_Behaviour의 범주별 Credit_Score
train.groupby(['Payment_Behaviour'])[['Credit_Score']].mean()

# High spent가 credit score가 더 높다
# 순서가 HL, HM, LL, HS, LM, LS 순이다.
# 크게 차이가 나지는 않지만 아무래도 차이가 있는 것으로 보이긴 한다.

# 라벨인코딩을 하되, 일정한 차이를 가지도록 하며
# 라벨인코딩 시 인코딩 후 민맥스 스케일링을 진행한다.
# High_spent_Large_value_payments : 3
# High_spent_Medium_value_payments : 2
# High_spent_Small_value_payments : 1
# Low_spent_Large_value_payments : 2
# Low_spent_Medium_value_payments : 1
# Low_spent_Medium_value_payments : 0

# 여기서는 어차피 2 언저리에서 큰 차이를 보이는 것 같지 않으니 드랍하기로 한다.
# 만약 성능이 안나오면 인코딩 코드를 진행하려한다.
df = pd.concat([X_train, X_test], axis = 0)
print(X_train.shape, X_test.shape, df.shape)
df = df.drop(columns = ['Payment_Behaviour'])
X_train, X_test = df[:train.shape[0]], df[train.shape[0]:]
print(X_train.shape, X_test.shape, df.shape)

(10000, 18) (10000, 18) (20000, 18)
(10000, 17) (10000, 17) (20000, 17)


In [27]:
# 각 범주에 따른 신용점수 평균
train.groupby(['Credit_Mix', 'Payment_of_Min_Amount'])[['Credit_Score']].mean()

Credit_Score
Credit_Mix Payment_of_Min_Amount              
Bad        NM                         1.396429
           Yes                        1.410621
Good       NM                         2.358639
           No                         2.316149
Standard   NM                         1.836257
           No                         2.017429
           Yes                        1.783544

In [28]:
# 'Payment_of_Min_Amount' 에 대해
# Yes와 No는 극명한 차이가 있음. 크레딧 믹스가 좋은 사람은 Yes가 없음. 최소한 매달 꼭 값으라고 정해진게 없다는 말인 거 같음.
# 'NM은 뭘까'
temp = ['Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Amount_invested_monthly', 'Monthly_Balance', 'Credit_Score', 'Credit_Mix', 'Payment_Behaviour', 'Age', 'Annual_Income', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Monthly_Inhand_Salary', 'Changed_Credit_Limit', 'Outstanding_Debt', 'Total_EMI_per_month']
train.groupby('Payment_of_Min_Amount')[temp].mean()


# 전체적으로 Yes 인 사람들에게 더 까가움. 나쁜 지표든 좋은 지표든 No 와의 차이보다 Yes 와의 차이가 더 큼
# 그렇다면 이렇게 할 수 있을 것으로 생각됨.
# Yes = 0
# NM = 0.75
# No = 1

# 그러나 이마저도, 다른 지표들로 어느정도 구분된다는 것 자체가 굳이 넣을 필요가 없다고 생각됨.
# 그리고 의미상 신용 등급 자체가 이 지표를 만들어낼 가능성이 높다고 생각함.
# drop
df = pd.concat([X_train, X_test], axis = 0)
print(X_train.shape, X_test.shape, df.shape)
df = df.drop(columns = ['Payment_of_Min_Amount'])
X_train, X_test = df[:train.shape[0]], df[train.shape[0]:]
print(X_train.shape, X_test.shape, df.shape)

(10000, 17) (10000, 17) (20000, 17)
(10000, 16) (10000, 16) (20000, 16)


/tmp/ipykernel_20/4228366315.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train.groupby('Payment_of_Min_Amount')[temp].mean()


In [29]:
# 'Credit_Mix'에 대하여 
# Bad 인 사람들은 NM과 Yes 만 있고 신용점수도 평균 1.5가 되지 않는다
# Good 인 사람들은 NM과 No 만 있고 신용점수가 평균 2.3이다
# Standard 인 사람들은 중간으로, NM, No, Yes 다 있으며, Yes 는 1.783, NM 은 1.836, No 는 2.017 로  약간의 차이를 보이고 있다.

# 만약 아래처럼 단적으로 본다면 차이가 극명하다. 거의 0.5씩 상승하는 것을 볼 수 있다.
train.groupby(['Credit_Mix'])[['Credit_Score']].mean()
# Credit_Mix Credit_Score
# Bad	1.408958
# Good	2.321523
# Standard	1.836201

# 따라서 이들은 라벨인코딩을 진행한다
# Bad : 0
# Standard : 1
# Good : 2

# 데이터 셋 합치기
df = pd.concat([X_train, X_test], axis = 0)

# 바꾸기 전 확인
print(X_train.shape, X_test.shape, df.shape)
print(df['Credit_Mix'].value_counts())
# 딕셔너리 만들어 대체
creditmix_dict = {'Bad': 0, 'Standard': 1, 'Good': 2}
df['Credit_Mix'] = df['Credit_Mix'].replace(creditmix_dict)
# 바꾼 후 확인
print(df['Credit_Mix'].replace(creditmix_dict).value_counts())

# 데이터셋 분리
X_train, X_test = df[:train.shape[0]], df[train.shape[0]:]
print(X_train.shape, X_test.shape, df.shape)

(10000, 16) (10000, 16) (20000, 16)
Standard    9181
Good        6100
Bad         4719
Name: Credit_Mix, dtype: int64
1    9181
2    6100
0    4719
Name: Credit_Mix, dtype: int64
(10000, 16) (10000, 16) (20000, 16)


In [30]:
# 트레인 데이터가 다 준비되었는지 보자.

# df = pd.concat([X_train, X_test], axis = 0)

# df.info()

# df.describe()

# 다 된 것 같다.

### 5. 타겟 데이터 확인

In [31]:
print(y_train.value_counts())

# 2    5237
# 1    2978
# 3    1785

# 어느정도 불균형이 있지만 쉽게 적은 데이터가 17%면 아주 작은 편도 아니기 때문에 분류를 바로 진행하도록 한다.

# 타깃 데이터 인코딩
target_encoding = {1:0, 2:1, 3:2}
y_train = y_train.replace(target_encoding)
print(y_train.value_counts())

2    5237
1    2978
3    1785
Name: Credit_Score, dtype: int64
1    5237
0    2978
2    1785
Name: Credit_Score, dtype: int64


## validation 데이터 나누기

In [32]:
from sklearn.model_selection import train_test_split

# help(train_test_split)
# train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 617)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(8000, 16) (2000, 16) (8000,) (2000,)


## 모델링

### 지표 관련 

macro_f1_score?

f1 score 에서 설정할 수 있음

In [33]:
# 모델 라이브러리
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRFClassifier

# 성능 지표
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix

 ```
 |  class_weight : {"balanced", "balanced_subsample"}, dict or list of dicts,             default=None
 |      Weights associated with classes in the form ``{class_label: weight}``.
 |      If not given, all classes are supposed to have weight one. For
 |      multi-output problems, a list of dicts can be provided in the same
 |      order as the columns of y.
 ```

In [34]:
# 랜덤포레스트1

# help(RandomForestClassifier)
rfc = RandomForestClassifier(class_weight = {0:0.3, 1:0.4, 2:0.3}, random_state = 617)
rfc.fit(X_tr, y_tr)
pred_rfc = rfc.predict(X_val)
pred_rfc

array([0, 2, 1, ..., 1, 1, 1])

In [35]:
# 랜덤포레스트1 검증

print('f1score: ', f1_score(y_val, pred_rfc, average = 'macro'))
print(confusion_matrix(y_val, pred_rfc))
# print('auc: ', roc_auc_score(y_val, pred_rfc, multi_class = 'ovo')) # 잘 안돼서 뺌
print('accuracy: ', accuracy_score(y_val, pred_rfc))


# 0.7149725785781892
# [[449 134  30]
#  [137 780 112]
#  [ 12 109 237]]
# accuracy:  0.733

f1score:  0.7149725785781892
[[449 134  30]
 [137 780 112]
 [ 12 109 237]]
accuracy:  0.733


In [36]:
# 랜덤포레스트2 : 클래스 가중치 없앴을 때

rfc = RandomForestClassifier(random_state = 617)
rfc.fit(X_tr, y_tr)
pred_rfc = rfc.predict(X_val)
pred_rfc

array([0, 1, 2, ..., 1, 1, 1])

In [37]:
# 랜덤포레스트2 검증

print('f1score: ', f1_score(y_val, pred_rfc, average = 'macro'))
print(confusion_matrix(y_val, pred_rfc))
print('accuracy: ', accuracy_score(y_val, pred_rfc))

# f1score:  0.7072079459233914
# [[444 136  33]
#  [143 778 108]
#  [ 12 115 231]]
# accuracy:  0.7265

f1score:  0.7072079459233914
[[444 136  33]
 [143 778 108]
 [ 12 115 231]]
accuracy:  0.7265


In [38]:
# 그래디언트부스트

gbc = GradientBoostingClassifier(random_state = 617)
gbc.fit(X_tr, y_tr)
pred_gbc = gbc.predict(X_val)
pred_gbc

array([0, 1, 2, ..., 1, 1, 1])

In [39]:
# 그래디언트부스트 검증 : 랜포랑 성능 비슷함 클래스 분류도 비슷하고

print('f1score: ', f1_score(y_val, pred_gbc, average = 'macro'))
print(confusion_matrix(y_val, pred_gbc))
print('accuracy: ', accuracy_score(y_val, pred_gbc))

# f1score:  0.7042427073254819
# [[415 158  40]
#  [123 778 128]
#  [ 13  95 250]]
# accuracy:  0.7215

f1score:  0.7042427073254819
[[415 158  40]
 [123 778 128]
 [ 13  95 250]]
accuracy:  0.7215


In [40]:
# XGBoost 

xgc = XGBClassifier(random_state = 617)
xgc.fit(X_tr, y_tr)
pred_xgc = xgc.predict(X_val)
pred_xgc

array([0, 1, 2, ..., 1, 1, 1])

In [41]:
# XGBoost 검증 : 성능이 더 떨어짐

print('f1score: ', f1_score(y_val, pred_xgc, average = 'macro'))
print(confusion_matrix(y_val, pred_xgc))
print('accuracy: ', accuracy_score(y_val, pred_xgc))

# f1score:  0.6900323497608474
# [[424 158  31]
#  [145 777 107]
#  [ 14 123 221]]
# accuracy:  0.711

f1score:  0.6900323497608474
[[424 158  31]
 [145 777 107]
 [ 14 123 221]]
accuracy:  0.711


In [42]:
# 추가로 XGBRFClassifier

xgrc = XGBRFClassifier(random_state = 617)
xgrc.fit(X_tr, y_tr)
pred_xgrc = xgrc.predict(X_val)
pred_xgrc

array([0, 2, 1, ..., 1, 1, 1])

In [43]:
# XGBRFClassifier 검증 : 분류라 그런지 트리계열이 성능이 더 낫다.

print('f1score: ', f1_score(y_val, pred_xgrc, average = 'macro'))
print(confusion_matrix(y_val, pred_xgrc))
print('accuracy: ', accuracy_score(y_val, pred_xgrc))

# f1score:  0.7119404631973677
# [[419 152  42]
#  [123 778 128]
#  [ 16  83 259]]
# accuracy:  0.728

f1score:  0.7119404631973677
[[419 152  42]
 [123 778 128]
 [ 16  83 259]]
accuracy:  0.728


### 모델 선택 : 랜덤포레스트
파라미터 튜닝

In [44]:
# 랜덤포레스트1-1
# class_weight = {0:0.2, 1:0.5, 2:0.3}

rfc = RandomForestClassifier(class_weight = {0:0.2, 1:0.5, 2:0.3}, random_state = 617)
rfc.fit(X_tr, y_tr)
pred_rfc = rfc.predict(X_val)
pred_rfc

array([0, 2, 2, ..., 1, 1, 1])

In [45]:
# 랜덤포레스트1-1 검증

print('f1score: ', f1_score(y_val, pred_rfc, average = 'macro'))
print(confusion_matrix(y_val, pred_rfc))
print('accuracy: ', accuracy_score(y_val, pred_rfc))

# f1score:  0.7164900913941885
# [[454 126  33]
#  [143 772 114]
#  [ 10 107 241]]
# accuracy:  0.7335

f1score:  0.7164900913941885
[[454 126  33]
 [143 772 114]
 [ 10 107 241]]
accuracy:  0.7335


In [46]:
# 랜덤포레스트1-2
# class_weight = {0:0.2, 1:0., 2:0.3}
# 분류 기준 수정해보기 : criterion='entropy'

rfc = RandomForestClassifier(criterion='entropy', class_weight = {0:0.2, 1:0.5, 2:0.3}, random_state = 617)
rfc.fit(X_tr, y_tr)
pred_rfc = rfc.predict(X_val)
pred_rfc

array([0, 1, 2, ..., 1, 1, 1])

In [47]:
# 랜덤포레스트1-2 검증

print('f1score: ', f1_score(y_val, pred_rfc, average = 'macro'))
print(confusion_matrix(y_val, pred_rfc))
print('accuracy: ', accuracy_score(y_val, pred_rfc))

# f1score:  0.7115857431779062
# [[451 130  32]
#  [140 773 116]
#  [ 13 109 236]]
# accuracy:  0.73

f1score:  0.7115857431779062
[[451 130  32]
 [140 773 116]
 [ 13 109 236]]
accuracy:  0.73


In [48]:
# 랜덤포레스트1-3
# class_weight = {0:0.2, 1:0., 2:0.3}
# 분류 기준 수정해보기 : criterion='log_loss'

rfc = RandomForestClassifier(criterion='log_loss', class_weight = {0:0.2, 1:0.5, 2:0.3}, random_state = 617)
rfc.fit(X_tr, y_tr)
pred_rfc = rfc.predict(X_val)
pred_rfc

array([0, 1, 2, ..., 1, 1, 1])

In [49]:
# 랜덤포레스트1-3 검증

print('f1score: ', f1_score(y_val, pred_rfc, average = 'macro'))
print(confusion_matrix(y_val, pred_rfc))
print('accuracy: ', accuracy_score(y_val, pred_rfc))

# f1score:  0.7115857431779062
# [[451 130  32]
#  [140 773 116]
#  [ 13 109 236]]
# accuracy:  0.73

f1score:  0.7115857431779062
[[451 130  32]
 [140 773 116]
 [ 13 109 236]]
accuracy:  0.73


In [50]:
# 랜덤포레스트1-4
# class_weight = {0:0.2, 1:0., 2:0.3}
# 트리 맥스 깊이 정하기

rfc = RandomForestClassifier(max_depth = 33, class_weight = {0:0.2, 1:0.5, 2:0.3}, random_state = 617)
rfc.fit(X_tr, y_tr)
pred_rfc = rfc.predict(X_val)
pred_rfc

array([0, 2, 2, ..., 1, 1, 1])

In [51]:
# 랜덤포레스트1-4 검증
# 맥스 깊이가 41 그정도 됐었나보다. 33으로 수정하니 더 낫다.

print('f1score: ', f1_score(y_val, pred_rfc, average = 'macro'))
print(confusion_matrix(y_val, pred_rfc))
print('accuracy: ', accuracy_score(y_val, pred_rfc))

# f1score:  0.7237787361722421
# [[457 123  33]
#  [146 776 107]
#  [ 10 102 246]]
# accuracy:  0.7395

f1score:  0.7237787361722421
[[457 123  33]
 [146 776 107]
 [ 10 102 246]]
accuracy:  0.7395


## 모델 선택 및 적합
랜포1-4

In [52]:
# 랜덤포레스트1-4로 예측 진행

rfc = RandomForestClassifier(max_depth = 33, class_weight = {0:0.2, 1:0.5, 2:0.3}, random_state = 617)
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
pred

array([1, 0, 0, ..., 0, 0, 1])

디코딩 관련 swap 방법 빠르게 구글링 : 

`res = dict((v,k) for k,v in dic.items())`
(https://sseongju1.tistory.com/35)

`{v:k for k, v in sample_dict.items()}`
(https://hellodata.tistory.com/29)


In [53]:
# 라벨 디코딩

target_encoding = {1:0, 2:1, 3:2}
target_decoding = dict((v,k) for k,v in target_encoding.items())

result = pd.DataFrame({'pred':pred})
result = result.replace(target_decoding)
result.value_counts()

pred
2       5259
1       2854
3       1887
dtype: int64

## 제출

제출 방식 : test데이터로 예측한 class 1개 컬럼만 csv로 제출 (컬럼명 pred, 파일명 result.csv)

In [54]:
result.to_csv('result.csv', index = False)

In [55]:
# 확인
read = pd.read_csv('/kaggle/working/result.csv')
print(read.shape)
print(read.value_counts())

(10000, 1)
pred
2       5259
1       2854
3       1887
dtype: int64


the end

restart 후 run all time : 39.36초

필요없는 모델 주석처리하고 진행 : 

In [56]:
read

,pred
0,2
1,1
2,1
3,2
4,1
...,...
9995,2
9996,2
9997,1
9998,1
